# GPT-2 Song Generator
This project attempts to generate lyrics based on lyrics from other songs. 



*Run on Google Colab, do not rerun in Jupyter - files not available*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !dir
%cd /content/drive/My\ Drive/AML\ Proj
!dir

/content/drive/My Drive/AML Proj
AllSongs.csv  Merged_Lyrics.csv  samples	 songdata_1.csv  songdata_3.csv
content       models		 songdata_0.csv  songdata_2.csv


:Using Tensorflow 1 as v2 causes issues with gpt2_simple. Also need to revert Keras to 2.3.1 for compatibility with Tensorflow 1.15

In [3]:
# !pip install q keras==2.3.1
# # keras.__version__

In [4]:
# %tensorflow_version 1.x
import os
import pandas as pd

import tensorflow as tf
try:
  import gpt_2_simple as gpt2
except:
  !pip3 -q install gpt-2-simple
  import gpt_2_simple as gpt2

In [5]:
print(tf.__version__)

2.7.0


## Load in text data
Files have been preprocessed and uploaded to Google Drive.

In [6]:
dfs = []
name = "songdata_{}.csv"
for i in range(4):
  dfs.append(pd.read_csv(name.format(i)))

df = pd.concat(dfs).reset_index(drop=True)

# df = pd.read_csv('Merged_Lyrics.csv')

In [7]:
# df['Lyric'].apply(lambda x: x.split('.'))

Create a copy of the files on colaboratory's VM (most convenient way to read them with GPT-2).


In [8]:
if not os.path.exists('content'):
    os.makedirs('content')
  
pd.DataFrame({"lyrics": df['text']})\
    .to_csv(os.path.join('content', 'lyrics.csv'), index=False)

We then download the pretrained models we will be using.

In [9]:
for model_name in ["124M", "355M", "774M"]:  # Choose from ["124M","355M","774M"]
  if not os.path.isdir(os.path.join("models", model_name)):  
    gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/

In [10]:
# Poor performance
learning_rate = 0.0001
optimizer = 'adam' # adam or sgd
batch_size = 1
model_name = "124M" # has to match one downloaded locally
sess = gpt2.start_tf_sess()

# Running out of memory
# learning_rate = 0.0001
# optimizer = 'adam' # adam or sgd
# batch_size = 1
# model_name = "774M" # has to match one downloaded locally
# sess774 = gpt2.start_tf_sess()

# learning_rate = 0.0001
# optimizer = 'adam' # adam or sgd
# batch_size = 1
# model_name = "355M" # has to match one downloaded locally
# sess355 = gpt2.start_tf_sess()

In [11]:
gpt2.finetune(sess,
              'content/lyrics.csv',
              reuse = tf.compat.v1.AUTO_REUSE,
              model_name=model_name,
              sample_every=50,
              save_every=50,
              print_every=10,
              learning_rate=learning_rate,
              batch_size=batch_size,
              restore_from='latest',
              only_train_transformer_layers = True,
              accumulate_gradients = 1,
              steps=500)   # steps is max number of training steps

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


dataset has 23199731 tokens
Training...
[10 | 12.04] loss=2.09 avg=2.09
[20 | 19.92] loss=2.59 avg=2.34
[30 | 27.82] loss=2.29 avg=2.32
[40 | 35.75] loss=1.94 avg=2.23
[50 | 43.69] loss=1.53 avg=2.08
Saving checkpoint/run1/model-50
======== SAMPLE 1 ========
,

[b]O[o]n[e]r[i]h [b]O[o]n[e]r[i]h

For me  

I'm on a plane  
You can come back to your place  
But with my name  
You are my best friend  
There's no more I'm not  
 I'll live you as your only brother  
I will live you as your only friend  
  
I am so happy, so happy  
To have all of you on my hand as my friend  
I'm so happy  
To have both of you on my hand as my friend  
I'm so happy  
To have both of you on my hand as my friend  
I'm so happy  
I'm so happy  
To have both of you on my hand as my friend  
And I'm glad that I didn't have to hear that song  
Even though my head hurts  
I'm sure that I'm all in it for you  
It's just the love that I have  
I'm afraid, I can think of no other way to express  
  
  
<|endoftext|>


In [12]:
lst_results=gpt2.generate(
    sess,
    nsamples=10,
    temperature=0.8, # change me
    top_p=0.9, # Change me
    return_as_list=True,
    include_prefix=False
    )

In [13]:
for res in lst_results:
  print(res)
  print('\n -------//------ \n')

This is the story of the day I was born  
That was the night I saw you  
It was the night I wanted to be a part of  
I don't know what else to say  
But I do know  
That I would have been the part of a celebrity  
That never had to be  
And if I'd had the chance to see you  
Would have been the star of a movie  
That never would have had to be  
  
This is the story of the day I was born  
That was the night I saw you  
It was the night I wanted to be a part of  
I don't know what else to say  
But I do know  
That I would have been the star of a movie

<|endoftext|>
<|startoftext|>It was a dream of mine  
As a boy and a girl  
That was never meant to come true  
  
You were with me at the end of the day  
I told myself it was not true  
  
I thought you would be a good one  
  
I thought you would be a good one  
  
But it wasn't true  
I never believed you  
  
That you were a star that never would have been  
That never would have been

<|endoftext|>
<|startoftext|>I'm at a party to